In [1]:
import os

In [2]:
%pwd

'c:\\Users\\KEERTHI RAJ\\Desktop\\KEERTHI RAJ\\PROJECTS\\DLPROJECTS\\Chicken-Disease-Classification-MLOPS\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\KEERTHI RAJ\\Desktop\\KEERTHI RAJ\\PROJECTS\\DLPROJECTS\\Chicken-Disease-Classification-MLOPS'

## Update the Params

## Update the Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_directory: Path
    tensorboard_root_log_directory: Path
    checkpoint_model_filepath: Path

## Update the Configurations

In [6]:
from ChickenDiseaseClassifier.constants import *
from ChickenDiseaseClassifier.utils.common import readYaml, createDirectories

In [7]:
class ConfigurationManager:
    """
    All these values are taken from the config file
    """

    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = readYaml(config_filepath)
        self.params = readYaml(params_filepath)
        createDirectories([self.config.artifacts_root])

    def prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_checkpoint_directory = os.path.dirname(config.checkpoint_model_filepath)
        createDirectories([
            Path(model_checkpoint_directory),
            Path(config.tensorboard_root_log_directory)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_directory=Path(config.root_directory),
            tensorboard_root_log_directory=Path(config.tensorboard_root_log_directory),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

## Updating the Components

In [8]:
from urllib.request import urlopen
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class PrepareCallback:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_directoy = os.path.join(
            self.config.tensorboard_root_log_directory,
            f"tb_logs_at_{timestamp}"
        )
        
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_directoy)
    
    @property
    def _create_checkpoint_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
    
    def get_tb_checkpoint_callbacks(self):
        return [
            self._create_tb_callback,
            self._create_checkpoint_callbacks
        ]

## Update the Pipeline

In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_checkpoint_callbacks()
except Exception as e:
    raise e

[2024-09-11 00:06:33,103: INFO:common:yaml file: config\config.yaml loaded successfully]


[2024-09-11 00:06:33,119: INFO:common:yaml file: params.yaml loaded successfully]
[2024-09-11 00:06:33,122: INFO:common:Created Directory at artifacts]
[2024-09-11 00:06:33,124: INFO:common:Created Directory at artifacts\prepare_callbacks\checkpoint_directory]
[2024-09-11 00:06:33,127: INFO:common:Created Directory at artifacts\prepare_callbacks\tensorflow_log_directory]


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=artifacts\prepare_callbacks\checkpoint_directory\model.h5